In [1]:
import pandas as pd
import ast
import numpy as np
import os
import csv
from tqdm.auto import tqdm

# make dataset

In [2]:
def activate_list_in_dataframe(df):
    for i in df.columns:
        a = df[i].values[0]
        if type(a) == type("a"):
            if a.startswith("["):
                df[i] = df[i].apply(lambda x: ast.literal_eval(x))
    return df

In [4]:
def _box_clip(boxes_data):
    def box_clip(box_data):
        return np.clip(box_data, 0, 1)
    return np.array([box_clip(box_data) for box_data in boxes_data])

In [5]:
data_type = ["train","val","test"]

for dtype in data_type:
    df_path = f"C:/Users/rh987/OneDrive/document/GitHub/SIA-MOT/EDA/{dtype}_det_df.csv"
    data_path = f"D:visdrone/VisDrone2019-MOT-{dtype}/sequences/"
    
    df = pd.read_csv(df_path)
    df = activate_list_in_dataframe(df)
    df["img_path"] = df["img_path"].apply(lambda x: data_path+x)
    df["frame_index"] = df["frame_index"].apply(lambda x: str(x).zfill(7))
    df["x_cen"] = [df["x_cen"].values[idx]/df["width"].values[idx] for idx in range(len(df))]
    df["y_cen"] = [df["y_cen"].values[idx]/df["height"].values[idx] for idx in range(len(df))]
    df["b_width"] = [df["b_width"].values[idx]/df["width"].values[idx] for idx in range(len(df))]
    df["b_height"] = [df["b_height"].values[idx]/df["height"].values[idx] for idx in range(len(df))]
    df["x_cen"] = df["x_cen"].apply(lambda x: _box_clip(x))
    df["y_cen"] = df["y_cen"].apply(lambda x: _box_clip(x))
    df["b_width"] = df["b_width"].apply(lambda x: _box_clip(x))
    df["b_height"] = df["b_height"].apply(lambda x: _box_clip(x))
    files = df.img_path.values
    video_name = df.video_name.values
    frame_idx = df.frame_index.values
#     #####
#     output_path = f"D:/visdrone/dataset/images/"

#     for idx in tqdm(range(len(df))):
#         file_name = "_".join(files[idx].split("/")[-2:])
#         try:
#             os.rename(
#                 files[idx],
#                 output_path+file_name
#             )
#         except:
#             print("오류")
#     ##### 
    output_path = f"D:/visdrone/dataset/labels/"

    for idx in tqdm(range(len(df))):
        file_name = "_".join(files[idx].split("/")[-2:])
        file_name = file_name.split(".")[0]+".txt"
        f = open(output_path+file_name, 'w')

        obj_counts = len(df["x_cen"].values[idx])
        for obj in range(obj_counts):
            label = []
            label.append(str(df["classes"].values[idx][obj]))
            label.append(str(df["x_cen"].values[idx][obj]))
            label.append(str(df["y_cen"].values[idx][obj]))
            label.append(str(df["b_width"].values[idx][obj]))
            label.append(str(df["b_height"].values[idx][obj]))
            write_data = " ".join(label)+"\n"
            f.write(write_data)
    ######
    output_path = f"D:/visdrone/dataset/{dtype}.csv"

    with open(output_path, mode="w", newline="") as train_file:
        for idx in range(len(df)):
            file_name = "_".join(files[idx].split("/")[-2:]).split(".")[0]
            data = [file_name+".jpg", file_name+".txt"]
            writer = csv.writer(train_file)
            writer.writerow(data)

  0%|          | 0/24100 [00:00<?, ?it/s]

  0%|          | 0/2846 [00:00<?, ?it/s]

  0%|          | 0/6635 [00:00<?, ?it/s]